###Carga de la data

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# Para que la salida al visualizar tenga 300 filas
pd.set_option('display.max_rows', 300)

In [ ]:
def str_a_date(texto):
    return datetime.strptime(texto,'%Y-%m-%d').date()

def str_a_datetime(texto):
    return datetime.strptime(texto,'%Y-%m-%d %H:%M:%S')

In [ ]:
drive.mount('/content/drive')

conv = {'fecNacimiento': str_a_date, 'fecha': str_a_datetime}
df = pd.read_excel('/content/drive/My Drive/Proyecto de Análisis de Datos/PY01.xlsx', converters = conv)

drive.flush_and_unmount()

Mounted at /content/drive


Visualizamos los primeros registros

In [ ]:
df.head()

,cliente,fecNacimiento,sexo,fecha,monto,descuento
0,SID1,NaN,NaN,2020-07-08 08:50:34,250.0,0.0
1,SID1,NaN,NaN,2020-07-08 08:51:21,100.0,0.0
2,SID1,NaN,NaN,2020-07-08 08:52:18,100.0,0.0
3,SID1,NaN,NaN,2020-07-08 08:52:37,75.0,0.0
4,SID1,NaN,NaN,2020-07-08 08:52:59,350.0,0.0


Revisamos las cantidad de filas y columnas

In [ ]:
ncols, nrows = df.shape
print('La base de datos tiene {} filas y {} columnas.'.format(ncols, nrows))

La base de datos tiene 3414 filas y 6 columnas.


###Identificación de inconsistencias

Verificamos que existen registros con valores nulos:

In [ ]:
# Registros con NAN
df.isnull().sum()

cliente             0
fecNacimiento    2689
sexo             2687
fecha               0
monto               0
descuento           0
dtype: int64

Revisamos cada una de las variables de la base datos en búsqueda de inconsistencias.

In [ ]:
print(df.columns.to_list())

['cliente', 'fecNacimiento', 'sexo', 'fecha', 'monto', 'descuento']


####Variable `cliente`

Revisamos si la parte numérica de los valores que toma esta variable es consecutiva:

In [ ]:
ids_unicos = np.sort(list(map(lambda x: int(x[3:]),df['cliente'].unique())))

q_ids = ids_unicos[-1]

i = 0
ids_no_encontrados = []
for id in range(1, q_ids + 1):
    if id == ids_unicos[i]:
        i += 1
        continue
    else:
        ids_no_encontrados.append(id)

print('Los siguientes {} IDs de clientes no están presentes dentro rango de valores consecutivos entre el mínimo y el máximo valor de la parte numérica de la variable:'.format(len(ids_no_encontrados)))
for id in ids_no_encontrados:
    print(id)

Los siguientes 85 IDs de clientes no están presentes dentro rango de valores consecutivos entre el mínimo y el máximo valor de la parte numérica de la variable:
11
22
37
51
60
62
64
110
112
117
123
145
146
149
154
156
159
161
163
164
166
167
168
169
176
179
181
182
183
185
186
187
188
189
190
192
193
194
195
196
198
199
200
206
212
214
215
217
218
220
221
222
223
225
226
229
231
232
233
235
236
238
239
240
241
242
271
284
285
290
297
312
318
322
329
330
331
333
357
358
362
363
364
372
373


####Variable `fecNacimiento`

El formato en el que viene esta fecha es `yyyy-MM-dd`. Notamos que, al momento de aplicar los conversores en la carga del dataset, no saltó ningún error, por lo que descartamos que existan fechas inválidas.

Revisamos si hay valores vacíos.

In [ ]:
len(df[pd.isna(df['fecNacimiento'])])

2689

Tenemos una gran cantidad de valores vacíos en esta variable; sin embargo, la gran mayoría corresponden a clientes anónimos.

In [ ]:
df[pd.isna(df['fecNacimiento'])].groupby('cliente', as_index = False)['fecNacimiento'].size()

,cliente,size
0,SID1,2687
1,SID148,1
2,SID151,1


Revisamos los valores únicos de esta variable.

In [ ]:
def mostrar_unicos_fecNacimiento():
    for fecha in np.append(np.sort(df[pd.isna(df['fecNacimiento']) == False]['fecNacimiento'].unique()),(np.nan,)): 
        print(fecha)

In [ ]:
mostrar_unicos_fecNacimiento()

0001-08-06
0002-03-24
0013-08-16
0038-10-25
0044-11-17
0045-09-29
0047-11-19
0047-12-03
0049-08-05
0050-01-12
0050-03-06
0052-07-16
0053-08-29
0054-06-11
0055-09-13
0056-10-10
0058-02-19
0059-09-23
0060-02-27
0060-07-25
0060-09-20
0061-04-14
0061-05-10
0061-06-02
0062-03-01
0062-04-08
0064-11-10
0065-02-19
0065-08-10
0066-11-11
0067-03-18
0067-05-17
0067-06-26
0067-07-20
0067-08-28
0067-12-24
0068-03-02
0068-04-03
0068-08-14
0069-02-13
0069-07-20
0069-12-13
0070-01-05
0070-01-06
0070-02-06
0070-06-30
0070-08-15
0071-02-16
0071-02-28
0071-10-04
0072-04-11
0072-05-18
0072-06-24
0072-07-13
0073-05-20
0073-11-17
0074-06-24
0074-08-30
0075-10-11
0075-11-26
0076-01-02
0076-02-28
0076-04-02
0076-05-15
0076-07-05
0076-07-18
0077-09-22
0077-12-17
0078-03-19
0078-06-24
0078-06-30
0078-11-15
0079-03-29
0079-08-07
0079-09-20
0079-09-23
0080-08-25
0081-04-01
0081-06-06
0081-08-26
0081-12-20
0082-03-02
0083-01-16
0083-08-22
0084-01-30
0084-05-23
0086-06-18
0087-12-18
0088-05-02
0089-09-28
0090-05-15

Notamos que hay fechas de nacimiento con años menores a 100, para estos casos vamos a hacer correcciones.

El cliente `SID21` tiene una fecha de nacimiento que, sospechamos, es errónea, ya sea el año realmente 1913 o 2013.

In [ ]:
df[df['fecNacimiento'] == datetime(13,8,16).date()]

,cliente,fecNacimiento,sexo,fecha,monto,descuento
3356,SID21,0013-08-16,Femenino,2021-02-10 10:56:44,1750.0,75.0


In [ ]:
# aplicar corrección para este caso en particular

A aquellas fechas de nacimiento con año menor a 21 les aumentamos 2000

In [ ]:
fecha_obtencion_dataset = datetime(2021,2,12).date()
df.loc[df['fecNacimiento'] < fecha_obtencion_dataset.replace(year = fecha_obtencion_dataset.year - 2000),'fecNacimiento'] = [d.replace(year = d.year + 2000) for d in df.loc[df['fecNacimiento'] < fecha_obtencion_dataset.replace(year = fecha_obtencion_dataset.year - 2000)]['fecNacimiento']]

Por otro lado, a las mayores a 21 pero menores a 100, les aumentamos 1900

In [ ]:
df.loc[df['fecNacimiento'] < datetime(100,1,1).date(),'fecNacimiento'] = [d.replace(year = d.year + 1900) for d in df.loc[df['fecNacimiento'] < datetime(100,1,1).date()]['fecNacimiento']]

La fecha de nacimiento `1900-01-01` se debe a un error del sistema. Tenemos una cantidad considerable de clientes con este error.

In [ ]:
len(df[df['fecNacimiento'] == datetime(1900,1,1).date()]['cliente'].unique())

176

Los clientes con fecha de nacimiento errónea no tienen sexo vacío, por lo que no se les podría considerar como anónimos.

In [ ]:
len(df[(df['fecNacimiento'] == datetime(1900,1,1).date()) & (pd.isna(df['sexo']))]['cliente'].unique())

0

Reemplazamos estos casos erróneos por valor nulo.

In [ ]:
df.loc[df['fecNacimiento'] == datetime(1900,1,1).date(), 'fecNacimiento'] = np.nan

Revisamos nuevamente los valores únicos luego de haber aplicado las correcciones.

In [ ]:
mostrar_unicos_fecNacimiento()

1938-10-25
1944-11-17
1945-09-29
1947-11-19
1947-12-03
1949-08-05
1950-01-12
1950-03-06
1952-07-16
1953-08-29
1954-06-11
1955-09-13
1956-10-10
1958-02-19
1959-09-23
1960-02-27
1960-07-25
1960-09-20
1961-04-14
1961-05-10
1961-06-02
1962-03-01
1962-04-08
1964-11-10
1965-02-19
1965-08-10
1966-11-11
1967-03-18
1967-05-17
1967-06-26
1967-07-20
1967-08-28
1967-12-24
1968-03-02
1968-04-03
1968-06-24
1968-08-14
1969-02-13
1969-07-20
1969-12-13
1970-01-05
1970-01-06
1970-02-06
1970-06-30
1970-08-15
1971-02-16
1971-02-28
1971-10-04
1972-01-15
1972-04-11
1972-05-18
1972-06-24
1972-07-13
1973-05-20
1973-11-17
1974-06-24
1974-08-30
1975-10-11
1975-11-26
1976-01-02
1976-02-28
1976-04-02
1976-05-15
1976-07-05
1976-07-18
1977-09-22
1977-12-17
1978-03-19
1978-06-24
1978-06-30
1978-11-15
1979-03-29
1979-08-07
1979-09-20
1979-09-23
1979-10-24
1980-08-25
1981-04-01
1981-06-06
1981-08-26
1981-12-20
1982-03-02
1983-01-16
1983-08-22
1984-01-30
1984-05-23
1984-07-22
1986-04-28
1986-06-18
1987-12-18
1988-05-02

Revisamos la cantidad de valores únicos de la variable `fecNacimiento` por cliente:

In [ ]:
nacimiento = df.groupby('cliente')['fecNacimiento'].nunique()
nacimiento

cliente
SID1      0
SID10     1
SID100    1
SID101    0
SID102    1
SID103    1
SID104    1
SID105    1
SID106    0
SID107    1
SID108    1
SID109    0
SID111    0
SID113    1
SID114    1
SID115    0
SID116    0
SID118    0
SID119    0
SID12     1
SID120    1
SID121    0
SID122    0
SID124    1
SID125    0
SID126    0
SID127    0
SID128    0
SID129    0
SID13     1
SID130    0
SID131    1
SID132    1
SID133    1
SID134    1
SID135    1
SID136    1
SID137    1
SID138    1
SID139    1
SID14     1
SID140    1
SID141    1
SID142    1
SID143    1
SID144    1
SID147    0
SID148    0
SID15     1
SID150    0
SID151    0
SID152    0
SID153    0
SID155    0
SID157    0
SID158    0
SID16     1
SID160    0
SID162    0
SID165    0
SID17     1
SID170    0
SID171    0
SID172    0
SID173    0
SID174    0
SID175    0
SID177    0
SID178    0
SID18     1
SID180    1
SID184    0
SID19     1
SID191    0
SID197    0
SID2      1
SID20     1
SID201    1
SID202    0
SID203    0
SID204    0
SID205    0
SID207  

Cantidad de clientes con valor vacío en la variable `fecNacimiento`

In [ ]:
sum(nacimiento == 0)

179

Cantidad de clientes con un solo valor de `fecNacimiento`

In [ ]:
sum(nacimiento == 1)

115

Cantidad de clientes con más de un valor de `fecNacimiento`

In [ ]:
sum(nacimiento > 1)

0

####Variable `sexo`

Cantidad de valores únicos que puede tomar la variable `sexo`

In [ ]:
df['sexo'].nunique()

3

Notamos que el valor `Femenino` también ha sido registrado con minúscula:

In [ ]:
df['sexo'].unique()

array([nan, 'Femenino', 'Masculino', 'femenino'], dtype=object)

Estandarizamos los valores de la variable:

In [ ]:
df.loc[df['sexo'] == 'femenino','sexo'] = 'Femenino'

Verificamos valores únicos de la variable sexo por cliente

In [ ]:
sexo = df.groupby('cliente')['sexo'].nunique()
sexo

cliente
SID1      0
SID10     1
SID100    1
SID101    1
SID102    1
SID103    1
SID104    1
SID105    1
SID106    1
SID107    1
SID108    1
SID109    1
SID111    1
SID113    1
SID114    1
SID115    1
SID116    1
SID118    1
SID119    1
SID12     1
SID120    1
SID121    1
SID122    1
SID124    1
SID125    1
SID126    1
SID127    1
SID128    1
SID129    1
SID13     1
SID130    1
SID131    1
SID132    1
SID133    1
SID134    1
SID135    1
SID136    1
SID137    1
SID138    1
SID139    1
SID14     1
SID140    1
SID141    1
SID142    1
SID143    1
SID144    1
SID147    1
SID148    1
SID15     1
SID150    1
SID151    1
SID152    1
SID153    1
SID155    1
SID157    1
SID158    1
SID16     1
SID160    1
SID162    1
SID165    1
SID17     1
SID170    1
SID171    1
SID172    1
SID173    1
SID174    1
SID175    1
SID177    1
SID178    1
SID18     1
SID180    1
SID184    1
SID19     1
SID191    1
SID197    1
SID2      1
SID20     1
SID201    1
SID202    1
SID203    1
SID204    1
SID205    1
SID207  

Cantidad de clientes con NAN en la variable sexo

In [ ]:
sum(sexo == 0)

1

Cantidad de clientes con 1 sexo

In [ ]:
sum(sexo == 1)

293

Cantidad de clientes con más de 1 sexo

In [ ]:
sum(sexo > 1)

0

####Variable `fecha`

El formato en el que viene esta fecha es `yyyy-MM-dd hh:mm:ss`. Notamos que, al momento de aplicar los conversores en la carga del dataset, no saltó ningún error, por lo que descartamos que existan fechas inválidas.

Revisamos si hay valores nulos.

In [ ]:
len(df[pd.isna(df['fecha'])])

0

A diferencia de la otra variable de tipo fecha, no tenemos ningún vacío.

Revisamos, excluyendo la parte de la hora, los valores únicos de esta variable.

In [ ]:
def mostrar_unicos_fecha_sin_hora():
    for fecha in np.sort(df['fecha'].map(datetime.date).unique()): 
        print(fecha)

In [ ]:
mostrar_unicos_fecha_sin_hora()

2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-27
2020-07-29
2020-07-30
2020-07-31
2020-08-01
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-08
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-15
2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-31
2020-09-01
2020-09-02
2020-09-03
2020-09-04
2020-09-05
2020-09-18
2020-09-19
2020-09-25
2020-09-26
2020-09-28
2020-09-29
2020-09-30
2020-10-01
2020-10-02
2020-10-03
2020-10-05
2020-10-06
2020-10-07
2020-10-09
2020-10-10
2020-10-12
2020-10-13
2020-10-14
2020-10-15
2020-10-16
2020-10-17
2020-10-19
2020-10-20
2020-10-21
2020-10-22
2020-10-23
2020-10-24
2020-10-26
2020-10-27
2020-10-28
2020-10-29
2020-10-30
2020-10-31
2020-11-02
2020-11-03
2020-11-04
2020-11-05
2020-11-06
2020-11-07
2020-11-09

Las compras se han registrado desde julio del 2020 hasta febrero del 2021

In [ ]:
min_fecha = min(df['fecha'])
max_fecha = max(df['fecha'])
print('El mínimo valor en la variable fecha es "{}"'.format(min_fecha))
print('El máximo valor en la variable fecha es "{}"'.format(max_fecha))

El mínimo valor en la variable fecha es "2020-07-08 08:50:34"
El máximo valor en la variable fecha es "2021-02-12 17:01:13"


Revisamos si hubo días en los que no hay registro de ventas.

In [ ]:
q_dias = (max_fecha.date() - min_fecha.date()).days
calendario = [min_fecha.date() + timedelta(days = i) for i in range(q_dias)]
fecha_unicos_sin_hora = np.sort(df[pd.isna(df['fecha']) == False]['fecha'].map(datetime.date).unique())

i = 0
fecha_no_encontrada = []
for fecha_calendario in calendario:
    if fecha_calendario == fecha_unicos_sin_hora[i]:
        i += 1
        continue
    else:
        fecha_no_encontrada.append(fecha_calendario)

nombre_dias = {1: 'lunes', 2: 'martes', 3: 'miércoles', 4: 'jueves', 5: 'viernes', 6: 'sábado', 7: 'domingo'}
print('Las siguientes {} fechas no están presentes dentro del periodo de días consecutivos entre la mínima y la máxima fecha de la variable:'.format(len(fecha_no_encontrada)))
for fecha in fecha_no_encontrada:
    print(fecha, nombre_dias[datetime.isoweekday(fecha)])

Las siguientes 49 fechas no están presentes dentro del periodo de días consecutivos entre la mínima y la máxima fecha de la variable:
2020-07-12 domingo
2020-07-19 domingo
2020-07-26 domingo
2020-07-28 martes
2020-08-02 domingo
2020-08-09 domingo
2020-08-16 domingo
2020-08-23 domingo
2020-08-30 domingo
2020-09-06 domingo
2020-09-07 lunes
2020-09-08 martes
2020-09-09 miércoles
2020-09-10 jueves
2020-09-11 viernes
2020-09-12 sábado
2020-09-13 domingo
2020-09-14 lunes
2020-09-15 martes
2020-09-16 miércoles
2020-09-17 jueves
2020-09-20 domingo
2020-09-21 lunes
2020-09-22 martes
2020-09-23 miércoles
2020-09-24 jueves
2020-09-27 domingo
2020-10-04 domingo
2020-10-08 jueves
2020-10-11 domingo
2020-10-18 domingo
2020-10-25 domingo
2020-11-01 domingo
2020-11-08 domingo
2020-11-15 domingo
2020-11-22 domingo
2020-11-29 domingo
2020-12-06 domingo
2020-12-08 martes
2020-12-13 domingo
2020-12-20 domingo
2020-12-25 viernes
2020-12-27 domingo
2021-01-01 viernes
2021-01-03 domingo
2021-01-10 domingo
20

In [ ]:
def mostrar_unicos_fecha_time(timetuple_index):
    fechas = df['fecha'].to_list()
    attributes = map(lambda x: datetime.timetuple(x)[timetuple_index], fechas)
    attributes = np.unique(list(attributes))
    print('Tenemos {} valores únicos:'.format(len(attributes)))
    for elemento in attributes: 
        print(elemento)

Revisamos las horas únicas.

In [ ]:
mostrar_unicos_fecha_time(3)

Tenemos 15 valores únicos:
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


Revisamos los minutos únicos.

In [ ]:
mostrar_unicos_fecha_time(4)

Tenemos 60 valores únicos:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


Revisamos los segundos únicos.

In [ ]:
mostrar_unicos_fecha_time(5)

Tenemos 60 valores únicos:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


####Variable `monto`

Revisamos los montos únicos.

In [ ]:
np.sort(df['monto'].unique())

array([   45. ,    50. ,    75. ,    90. ,   100. ,   115. ,   125. ,
         150. ,   175. ,   180. ,   200. ,   225. ,   240. ,   250. ,
         275. ,   290. ,   300. ,   315. ,   325. ,   337.5,   350. ,
         360. ,   375. ,   400. ,   405. ,   425. ,   440. ,   450. ,
         475. ,   500. ,   525. ,   540. ,   550. ,   575. ,   600. ,
         625. ,   650. ,   675. ,   700. ,   720. ,   725. ,   750. ,
         760. ,   775. ,   800. ,   825. ,   850. ,   855. ,   875. ,
         900. ,   925. ,   950. ,   975. ,  1000. ,  1025. ,  1050. ,
        1075. ,  1100. ,  1125. ,  1145. ,  1150. ,  1175. ,  1190. ,
        1200. ,  1215. ,  1225. ,  1250. ,  1275. ,  1300. ,  1305. ,
        1325. ,  1350. ,  1375. ,  1400. ,  1425. ,  1450. ,  1475. ,
        1500. ,  1525. ,  1550. ,  1575. ,  1600. ,  1620. ,  1625. ,
        1650. ,  1675. ,  1700. ,  1725. ,  1750. ,  1775. ,  1800. ,
        1825. ,  1850. ,  1875. ,  1890. ,  1900. ,  1910. ,  1925. ,
        1950. ,  197

In [ ]:
print('El mínimo monto es {}'.format(min(df['monto'])))
print('El máximo monto es {}'.format(max(df['monto'])))

El mínimo monto es 45.0
El máximo monto es 18125.0


No hay ningún valor negativo, tampoco hay monto 0. Concluimos que no hay inconsistencias en esta variable.

####Variable `descuento`

Revisamos los descuentos únicos.

In [ ]:
np.sort(df['descuento'].unique())

array([   0. ,    5. ,   10. ,   20. ,   25. ,   35. ,   37.5,   40. ,
         45. ,   50. ,   60. ,   75. ,   80. ,   95. ,  100. ,  125. ,
        130. ,  135. ,  145. ,  150. ,  175. ,  180. ,  200. ,  210. ,
        225. ,  240. ,  250. ,  275. ,  300. ,  320. ,  340. ,  350. ,
        400. ,  405. ,  450. ,  465. ,  500. ,  602.5,  607.5,  650. ,
        700. ,  800. ,  905. , 1000. , 1200. , 1400. , 1700. ])

In [ ]:
print('El mínimo monto es {}'.format(min(df['descuento'])))
print('El máximo monto es {}'.format(max(df['descuento'])))

El mínimo monto es 0.0
El máximo monto es 1700.0


No hay ningún valor negativo. Concluimos que no hay inconsistencias en esta variable.

####Adicionales

Revisamos si hay compras donde el descuento sea mayor al monto.

In [ ]:
df[df['descuento'] > df['monto']]

,cliente,fecNacimiento,sexo,fecha,monto,descuento
86,SID7,1950-01-12,Femenino,2020-07-10 16:53:28,1000.0,1200.0
1042,SID23,2001-08-06,Femenino,2020-08-27 10:44:32,700.0,800.0
1931,SID259,NaN,Femenino,2020-11-21 07:58:57,250.0,300.0


Notamos que hay 3 registros donde el descuento es mayor al monto de compra, dando como resultado que el monto final sea negativo.

In [ ]:
#df.to_csv('salida.csv')

In [ ]:
df[["h","m","s"]] = pd.DataFrame([(x.hour, x.minute, x.second) for x in df["fecha"]])

In [ ]:
df.head()

,cliente,fecNacimiento,sexo,fecha,monto,descuento,h,m,s
0,SID1,NaN,NaN,2020-07-08 08:50:34,250.0,0.0,8,50,34
1,SID1,NaN,NaN,2020-07-08 08:51:21,100.0,0.0,8,51,21
2,SID1,NaN,NaN,2020-07-08 08:52:18,100.0,0.0,8,52,18
3,SID1,NaN,NaN,2020-07-08 08:52:37,75.0,0.0,8,52,37
4,SID1,NaN,NaN,2020-07-08 08:52:59,350.0,0.0,8,52,59


In [ ]:
df[df['h']>18]

,cliente,fecNacimiento,sexo,fecha,monto,descuento,h,m,s
105,SID1,NaN,NaN,2020-07-11 19:02:46,525.0,0.0,19,2,46
106,SID1,NaN,NaN,2020-07-11 19:04:02,3100.0,0.0,19,4,2
107,SID6,1983-08-22,Femenino,2020-07-11 19:06:00,2650.0,700.0,19,6,0
108,SID1,NaN,NaN,2020-07-11 20:17:40,400.0,0.0,20,17,40
693,SID1,NaN,NaN,2020-08-08 19:00:40,1450.0,0.0,19,0,40
694,SID1,NaN,NaN,2020-08-08 19:02:16,2950.0,0.0,19,2,16
695,SID1,NaN,NaN,2020-08-08 19:02:55,725.0,0.0,19,2,55
696,SID1,NaN,NaN,2020-08-08 19:03:39,1300.0,0.0,19,3,39
697,SID13,1974-06-24,Femenino,2020-08-08 19:04:25,675.0,25.0,19,4,25
698,SID1,NaN,NaN,2020-08-08 19:05:15,575.0,0.0,19,5,15


In [ ]:
len(df[df['h']>18])

11

In [ ]:
df[df['h']<8]

,cliente,fecNacimiento,sexo,fecha,monto,descuento,h,m,s
1627,SID1,NaN,NaN,2020-10-27 07:13:12,1025.0,0.0,7,13,12
1628,SID1,NaN,NaN,2020-10-27 07:13:23,350.0,0.0,7,13,23
1629,SID1,NaN,NaN,2020-10-27 07:13:45,375.0,0.0,7,13,45
1630,SID2,1972-01-15,Femenino,2020-10-27 07:14:33,750.0,25.0,7,14,33
1631,SID1,NaN,NaN,2020-10-27 07:14:57,350.0,0.0,7,14,57
1632,SID1,NaN,NaN,2020-10-27 07:15:18,650.0,0.0,7,15,18
1633,SID1,NaN,NaN,2020-10-27 07:15:28,175.0,0.0,7,15,28
1634,SID1,NaN,NaN,2020-10-27 07:53:22,150.0,0.0,7,53,22
1645,SID1,NaN,NaN,2020-10-28 07:39:02,350.0,0.0,7,39,2
1646,SID1,NaN,NaN,2020-10-28 07:39:23,300.0,0.0,7,39,23


In [ ]:
len(df[df['h']<8])

99

In [ ]:
df2 = df.drop(df[df['cliente']=='SID1'].index)
tabla01=df2.pivot_table(index='cliente',values='monto',aggfunc="sum") 

In [ ]:
def funcion (clientes):
#clientes= tabla01.nlargest(3,'monto',keep='all')
  topmas=tabla01.nlargest(clientes,'monto',keep='all')
  print(topmas)

In [ ]:
import ipywidgets as widgets

x= widgets.IntSlider(

    min=1,
    max=20,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [ ]:
widgets.interact(funcion,clientes=x)

interactive(children=(IntSlider(value=1, continuous_update=False, description='Test:', max=20, min=1), Output(…

<function __main__.funcion>